In [1]:
# Average 60+% accuracy

# -*- coding: utf-8 -*-
"""CNN.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1RDQrJejOfArNgzjcXMDQ0C4bBzMljDtA

# Connect with Google Drive
"""

# from google.colab import drive
# drive.mount('/content/drive')

"""# Importing Libraries"""
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import scipy
from scipy.interpolate import interp1d
from scipy.stats import kurtosis
from scipy.stats import skew
from sklearn import preprocessing
from sklearn.preprocessing import normalize
from scipy.ndimage import zoom
from skimage.measure import block_reduce
from skimage.transform import resize
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.optimizers import Adam
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

"""# Loading Data"""

path = "../../Cog_DataSets/testing/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/testing/"
test_acc = np.load(path+"testAccelerometer.npy")
test_grav = np.load(path+"testGravity.npy")
test_gyro = np.load(path+"testGyroscope.npy")
test_jinsAcc = np.load(path+"testJinsAccelerometer.npy")
test_jinsGyro = np.load(path+"testJinsGyroscope.npy")
test_Label =np.load(path+"testLabels.npy") 
test_linAcc = np.load(path+"testLinearAcceleration.npy")
test_MsAcc = np.load(path+"testMSAccelerometer.npy")
test_MsGyro = np.load(path + "testMSGyroscope.npy")
test_MsMag = np.load(path+"testMagnetometer.npy")
# test_acc

path = "../../Cog_DataSets/training/"
# path = "drive/MyDrive/Colab Notebooks/CogAge/Datasets/training/"
train_acc = np.load(path+"trainAccelerometer.npy")
train_grav = np.load(path+"trainGravity.npy")
train_gyro = np.load(path+"trainGyroscope.npy")
train_jinsAcc = np.load(path+"trainJinsAccelerometer.npy")
train_jinsGyro = np.load(path+"trainJinsGyroscope.npy")
train_Label =np.load(path+"trainLabels.npy") 
train_linAcc = np.load(path+"trainLinearAcceleration.npy")
train_MsAcc = np.load(path+"trainMSAccelerometer.npy")
train_MsGyro = np.load(path + "trainMSGyroscope.npy")
train_MsMag = np.load(path+"trainMagnetometer.npy")
# print(train_Label.shape)


def Normalize(X):
  norm = []
  for I in range(len(X)):
    norm.append(normalize(X[I]))
  norm=np.array(norm)
  return norm

train_acc = Normalize(train_acc)
train_gyro = Normalize(train_gyro)
train_grav = Normalize(train_grav)
train_linAcc = Normalize(train_linAcc)
train_MsMag = Normalize(train_MsMag)
train_MsAcc = Normalize(train_MsAcc)
train_MsGyro = Normalize(train_MsGyro)
train_jinsAcc = Normalize(train_jinsAcc)
train_jinsGyro = Normalize(train_jinsGyro)

test_acc = Normalize(test_acc)
test_gyro = Normalize(test_gyro)
test_grav = Normalize(test_grav)
test_linAcc = Normalize(test_linAcc)
test_MsMag = Normalize(test_MsMag)
test_MsAcc = Normalize(test_MsAcc)
test_MsGyro = Normalize(test_MsGyro)
test_jinsAcc = Normalize(test_jinsAcc)
test_jinsGyro = Normalize(test_jinsGyro)



"""# Reshape and stack Data Before Fitting to Model"""
# changing shape of sensor data to (#,400,3)
# downsample
train_acc_reshaped = block_reduce(train_acc, block_size=(1, 2, 1), func=np.mean)
train_gyro_reshaped = block_reduce(train_gyro, block_size=(1, 2, 1), func=np.mean)
train_grav_reshaped = block_reduce(train_grav, block_size=(1, 2, 1), func=np.mean)
train_linAcc_reshaped = block_reduce(train_linAcc, block_size=(1, 2, 1), func=np.mean)
# upsample
train_MsAcc_reshaped = resize(train_MsAcc, (2284, 400, 3), mode='edge')
train_MsGyro_reshaped = resize(train_MsGyro, (2284, 400, 3), mode='edge')
# upsample
train_MsMag_reshaped = np.repeat(train_MsMag, 2, axis=1)
train_jinsAcc_reshaped = np.repeat(train_jinsAcc, 5, axis=1)
train_jinsGyro_reshaped = np.repeat(train_jinsGyro, 5, axis=1)

test_acc_reshaped = block_reduce(test_acc, block_size=(1, 2, 1), func=np.mean)
test_gyro_reshaped = block_reduce(test_gyro, block_size=(1, 2, 1), func=np.mean)
test_grav_reshaped = block_reduce(test_grav, block_size=(1, 2, 1), func=np.mean)
test_linAcc_reshaped = block_reduce(test_linAcc, block_size=(1, 2, 1), func=np.mean)
test_MsAcc_reshaped = resize(test_MsAcc, (2288, 400, 3), mode='edge')
test_MsGyro_reshaped = resize(test_MsGyro, (2288, 400, 3), mode='edge')
test_MsMag_reshaped = np.repeat(test_MsMag, 2, axis=1)
test_jinsAcc_reshaped = np.repeat(test_jinsAcc, 5, axis=1)
test_jinsGyro_reshaped = np.repeat(test_jinsGyro, 5, axis=1)

# all data of shape #,400,3
# adding all relative data.
# Mobile training accelerometer + Mobile testing accelerometer data
train_acc_reshaped = np.append(train_acc_reshaped,test_acc_reshaped, axis=0)
train_gyro_reshaped = np.append(train_gyro_reshaped,test_gyro_reshaped, axis=0)
train_grav_reshaped = np.append(train_grav_reshaped,test_grav_reshaped, axis=0)
train_linAcc_reshaped = np.append(train_linAcc_reshaped,test_linAcc_reshaped, axis=0)
train_MsAcc_reshaped = np.append(train_MsAcc_reshaped,test_MsAcc_reshaped, axis=0)
train_MsGyro_reshaped = np.append(train_MsGyro_reshaped,test_MsGyro_reshaped, axis=0)
train_MsMag_reshaped = np.append(train_MsMag_reshaped,test_MsMag_reshaped, axis=0)
train_jinsAcc_reshaped = np.append(train_jinsAcc_reshaped,test_jinsAcc_reshaped, axis=0)
train_jinsGyro_reshaped = np.append(train_jinsGyro_reshaped,test_jinsGyro_reshaped, axis=0)


print("Shape of all sensors after up/down sample... ", train_acc_reshaped.shape, train_gyro_reshaped.shape, train_grav_reshaped.shape, train_linAcc_reshaped.shape
                       , train_MsAcc_reshaped.shape, train_MsGyro_reshaped.shape, train_MsMag_reshaped.shape,
                       train_jinsAcc_reshaped.shape, train_jinsGyro_reshaped.shape)

# all data of shape 4572,400,3
# 4572 = 2284(training) + 2288(testing)


# stack
all_data = np.stack([train_acc_reshaped, train_gyro_reshaped, train_grav_reshaped, train_linAcc_reshaped
                       , train_MsAcc_reshaped, train_MsGyro_reshaped, train_MsMag_reshaped,
                       train_jinsAcc_reshaped, train_jinsGyro_reshaped], axis=-1)

all_Label = np.append(train_Label, test_Label, axis=-1)


2023-04-27 23:19:13.567024: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 23:19:13.623481: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-27 23:19:13.624448: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 23:19:14.458516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Shape of all sensors after up/down sample...  (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3) (4572, 400, 3)


In [2]:

# # 90% training data + labels
# train_data = all_data[: int(all_data.shape[0]*0.9)]
# # 10% testing data + labels
# test_data = all_data[int(all_data.shape[0]*0.9):]
# train_labels = all_Label[: int(all_Label.shape[0]*0.9)]
# test_labels = all_Label[int(all_Label.shape[0]*0.9):]

# print("\nShape of training and testin data + labels...\n")
# print(train_data.shape, test_data.shape)
# print(train_labels.shape, test_labels.shape)


# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(all_data, all_Label, test_size=0.2, random_state=42)

print("Shape of training + validation Data")
print(np.shape(x_train), np.shape(x_val))

# y_train = to_categorical(y_train, num_classes=55)
# y_val = to_categorical(y_val, num_classes=55)

print("Shape of training + validation Labels")
print(np.shape(y_train), np.shape(y_val))





# Reshape the input data to (3291, 400*3*9)
# x_train = np.reshape(x_train, (x_train.shape[0], -1))
train_data = np.reshape(x_train, (x_train.shape[0],-1))
test_data = np.reshape(x_val, (x_val.shape[0],-1))
input_shape=(400*3*9,)

print("reshape: ", train_data.shape)
print("reshape: ", test_data.shape)


Shape of training + validation Data
(3657, 400, 3, 9) (915, 400, 3, 9)
Shape of training + validation Labels
(3657,) (915,)
reshape:  (3657, 10800)
reshape:  (915, 10800)


In [3]:
# HYPER PARAMTERS

# Filter parameters, i.e. about the number of inputs processed by each neuron of the convolutional layer
filter1Size = (11,1)
filter2Size = (13,1)
filter3Size = (13,1)

# Downsampling factors of the pooling layers
poolingLayer1Factor = (2,1)
poolingLayer2Factor = (3,1)
poolingLayer3Factor = (2,1)

# Number of feature maps processed by each convolutional layer
nbFeaturesLayer1 = 50
nbFeaturesLayer2 = 40
nbFeaturesLayer3 = 30

# Activation function of the convolutional layer(s)
activationConv = 'relu'

# Output dimension of the LSTM
outputLSTM = 100

# Parameters of the dense layer
activationMLP = 'relu'
inputMLP = 2500

# Training parameters
batchSize = 200
numberOfEpochs = 10
learningRate = 0.001

# input_shape = (400,3,9)
nbClasses = 55

#-------------------------------------------------------------------------------------------------------
# autoencoder2: define a autoencoder with 2 hidden layers
#-------------------------------------------------------------------------------------------------------
def autoencoder2(
    inputShape,
    inputMLP1,
    inputMLP2,
    activationMLP,
    decoder=True):

    model = Sequential()

    # Dense layer 1 of the encoder
    model.add(Dense(inputMLP1,input_shape=inputShape,activation=activationMLP))
    print('Encoder dense 1 shape: ------------------------------')
    print(model.layers[-1].output_shape)

    # Dense layer 2 of the encoder
    model.add(Dense(inputMLP2,activation=activationMLP))
    print('Encoder dense 2 shape: ------------------------------')
    print(model.layers[-1].output_shape)


    if decoder:

        # Dense layer 1 of the decoder
        model.add(Dense(inputMLP2,activation=activationMLP))
        print('Decoder dense 1 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Dense layer 2 of the decoder
        model.add(Dense(inputMLP1,activation=activationMLP))
        print('Decoder dense 2 shape: ------------------------------')
        print(model.layers[-1].output_shape)

        # Output layer
        model.add(Dense(inputShape[0],activation='linear')) 
        print('Output shape: ------------------------------')
        print(model.layers[-1].output_shape)      

    return model

model = autoencoder2(inputShape=input_shape,
                          inputMLP1=inputMLP,
                          inputMLP2=inputMLP,
                          activationMLP=activationMLP)


Encoder dense 1 shape: ------------------------------
(None, 2500)
Encoder dense 2 shape: ------------------------------
(None, 2500)
Decoder dense 1 shape: ------------------------------
(None, 2500)
Decoder dense 2 shape: ------------------------------
(None, 2500)
Output shape: ------------------------------
(None, 10800)


In [4]:
model.compile(
    loss='mse',
    optimizer=Adam(learning_rate=learningRate),
    metrics=['accuracy']
)


model.fit(x=train_data, y=train_data, epochs=2, batch_size=batchSize, validation_data=(test_data, test_data))


Epoch 1/2
19/19 [==============================] - 27s 1s/step - loss: 0.2210 - accuracy: 0.0014 - val_loss: 0.1642 - val_accuracy: 0.0011
Epoch 2/2
19/19 [==============================] - 28s 2s/step - loss: 0.1513 - accuracy: 8.2034e-04 - val_loss: 0.1407 - val_accuracy: 0.0011


In [5]:
model.summary()

from sklearn.svm import SVC

x_train_compressed = model.predict(train_data)
x_test_compressed = model.predict(test_data)

# Train an SVM classifier on the compressed data
svm = SVC(kernel='linear')
svm.fit(x_train_compressed, y_train)

# print(x_train_comp.shape)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2500)              27002500  
                                                                 
 dense_1 (Dense)             (None, 2500)              6252500   
                                                                 
 dense_2 (Dense)             (None, 2500)              6252500   
                                                                 
 dense_3 (Dense)             (None, 2500)              6252500   
                                                                 
 dense_4 (Dense)             (None, 10800)             27010800  
                                                                 
Total params: 72,770,800
Trainable params: 72,770,800
Non-trainable params: 0
_________________________________________________________________
29/29 [==============================] - 3s 

SVC(kernel='linear')

In [6]:
# Evaluate the accuracy of the SVM classifier
svm_score = svm.score(x_test_compressed, y_val)
print("Accuracy: %.2f%%" % (svm_score*100))


Accuracy: 54.21%
